In [ ]:
import openpyxl
import formulas
from formulas.tokens import Token
import re

tokens, builder = formulas.Parser().ast("=(1 + 1) + handred / A2")
path = "incrementation excel example.xlsx"
wb = openpyxl.load_workbook(path)
ws = wb.active

match_group = []

def split_alpha_num(s):
    m = re.match(r"([A-Za-z]+)(\d*)", s)  # number part optional
    if m:
        return m.group(1), m.group(2) or None  # return None if no digits
    return None, None

def does_match(t1: list[Token], t2: list[Token]):
    if len(t1) != len(t2):
        return False

    for right_token, left_token in zip(t1,t2):
        if right_token.__class__ is not left_token.__class__:
            return False
        if isinstance(right_token)

for row in ws.iter_rows(values_only=True):
    for cell in row:
        if isinstance(cell, str) and cell.startswith("="):
            group_detected = False
            new_tokens, _ = formulas.Parser().ast(cell)
            for group in match_group:
                sample = group[0]
                if does_match(new_tokens, sample):
                    group.append(new_tokens)
                    group_detected = True
                    break
            if not group_detected:
                match_group.append([new_tokens])

=A2^power
=A2^K1
=A2^$K$1
=A3^power
=A3^K2
=B3^L2
=C3^M2
=D3^N2
=E3^O2
=A4^power
=A4^K3
=A4^$K$1
=A5^power
=A5^K4
=A5^$K$1
=A6^power
=A6^K5
=A6^$K$1
=A7^power
=A7^K6
=A7^$K$1
=A8^power
=A8^K7
=A10^$K$1
=A9^power
=A9^K8
=A11^$K$1
=A10^power
=A10^K9
=A12^$K$1
=A11^power
=A11^K10
=A13^$K$1
=A12^power
=A12^K11
=A14^$K$1
=A13^power
=A13^K12
=A15^$K$1
=A14^power
=A14^K13
=A16^$K$1


In [29]:
tokens, builder = formulas.Parser().ast("=(1 + 1) + handred / A2")
tokens[6].__class__

formulas.tokens.operand.Range

In [ ]:
import re



print(split_alpha_num("hello123"))  # ('hello', '123')
print(split_alpha_num("hello"))     # ('hello', None)


('hello', '123')
('hello', None)


In [25]:
str is str

True